# Enumerate binary alloy structures

In [1]:
from itertools import combinations

import numpy as np
from ase.build import bulk
from ase.data import chemical_symbols, reference_states
from ase.db import connect
from ase.io import read
from calorine.calculators import CPUNEP
from calorine.nep import read_model
from calorine.tools import relax_structure
from icet.tools import enumerate_structures
from pandas import DataFrame

## Load model to retrieve supported species

In [2]:
model_filename = '../model/nep.txt'
model = read_model(model_filename)

## Collect ground state crystal structure for supported species

In [3]:
reference_structures = {}
for species, refstate in zip(chemical_symbols, reference_states):
    if species in model.types:
        reference_structures[species] = refstate['symmetry']

## Relax structures

In [4]:
data = []
structures = {}
for species in model.types:
    structures[species] = {}
    ref_cs = reference_structures[species]
    for cs in ['fcc', 'bcc', 'hcp', 'omega']:
        if cs == 'omega':
            prim = read('omega-structure.xyz')
            prim.symbols = species
        else:
            kwargs = dict()
            if cs != ref_cs:
                kwargs['crystalstructure'] = cs
                kwargs['a'] = 3.5
                if cs == 'hcp':
                    kwargs['c'] = 4.5
            prim = bulk(species, **kwargs)

        prim.calc = CPUNEP(model_filename)
        relax_structure(prim, fmax=1e-5)
        
        lats = np.linalg.norm(prim.cell, axis=1)
        data.append(dict(
            species = species,
            cs = cs,
            energy = prim.get_potential_energy() / len(prim),
            volume = prim.get_volume() / len(prim),
            a = lats[0],
            c_to_a_ratio = lats[2] / lats[0],
        ))

        structures[species][cs] = prim

df_structures = DataFrame.from_dict(data)

## Review results

In [5]:
for species, df in df_structures.groupby('species'):
    df.sort_values('energy', inplace=True)
    df.energy -= df.energy.min()
    #display(df)

In [6]:
nmax = dict(
    bcc=5,
    fcc=5,
    hcp=3,
    omega=2,
)

db = connect('enumerate-structures.db', append=False)

for A, B in combinations(model.types, 2):

    for cs in nmax:
        
        if not (cs == 'omega' and (A in 'TiZr' or B in 'TiZr') and (reference_structures[A] == 'hcp' or reference_structures[B] == 'hcp')) and \
                not (reference_structures[A] == cs or reference_structures[B] == cs):
            continue
        print(f'{A:2}  {B:2}  {cs:5}')

        prim_A = structures[A][cs]
        prim_B = structures[B][cs]
        vol_A = prim_A.get_volume()
        vol_B = prim_B.get_volume()

        allowed_symbols = [[A, B] for s in prim_A]
        n = 0
        for k, s in enumerate(enumerate_structures(
                prim_A, chemical_symbols=allowed_symbols, sizes=range(1, nmax[cs]+1))):
            n += 1
            conc_A = s.symbols.count(A) / len(s)
            conc_B = s.symbols.count(B) / len(s)

            vol = vol_A * conc_A + vol_B * conc_B
            vol0 = s.get_volume() / len(s)
            scale = (vol / vol0) ** (1 / 3)
            cell = s.cell[:] * scale
            s.set_cell(cell, scale_atoms=True)

            sname = f'{cs}-ix{k}-{s.get_chemical_formula()}'
            try:
                _ = db.get(name=sname)
            except KeyError:
                db.write(s, name=sname)

Ag  Al  fcc  
Ag  Au  fcc  
Ag  Cr  bcc  
Ag  Cr  fcc  
Ag  Cu  fcc  
Ag  Mg  fcc  
Ag  Mg  hcp  
Ag  Mo  bcc  
Ag  Mo  fcc  
Ag  Ni  fcc  
Ag  Pb  fcc  
Ag  Pd  fcc  
Ag  Pt  fcc  
Ag  Ta  bcc  
Ag  Ta  fcc  
Ag  Ti  fcc  
Ag  Ti  hcp  
Ag  Ti  omega
Ag  V   bcc  
Ag  V   fcc  
Ag  W   bcc  
Ag  W   fcc  
Ag  Zr  fcc  
Ag  Zr  hcp  
Ag  Zr  omega
Al  Au  fcc  
Al  Cr  bcc  
Al  Cr  fcc  
Al  Cu  fcc  
Al  Mg  fcc  
Al  Mg  hcp  
Al  Mo  bcc  
Al  Mo  fcc  
Al  Ni  fcc  
Al  Pb  fcc  
Al  Pd  fcc  
Al  Pt  fcc  
Al  Ta  bcc  
Al  Ta  fcc  
Al  Ti  fcc  
Al  Ti  hcp  
Al  Ti  omega
Al  V   bcc  
Al  V   fcc  
Al  W   bcc  
Al  W   fcc  
Al  Zr  fcc  
Al  Zr  hcp  
Al  Zr  omega
Au  Cr  bcc  
Au  Cr  fcc  
Au  Cu  fcc  
Au  Mg  fcc  
Au  Mg  hcp  
Au  Mo  bcc  
Au  Mo  fcc  
Au  Ni  fcc  
Au  Pb  fcc  
Au  Pd  fcc  
Au  Pt  fcc  
Au  Ta  bcc  
Au  Ta  fcc  
Au  Ti  fcc  
Au  Ti  hcp  
Au  Ti  omega
Au  V   bcc  
Au  V   fcc  
Au  W   bcc  
Au  W   fcc  
Au  Zr  fcc  
Au  Zr  hcp  
Au  Zr